In [1]:
def inMemory_database_query(queries):
    database = {}
    results = []

    for query in queries:
        command = query[0]
        key = query[1]
        field = query[2]

        if command == "SET_OR_INCREMENT":
            value = int(query[3])
            if key not in database:
                database[key] = {}
            if field in database[key]:
                database[key][field] += value
            else:
                database[key][field] = value
            results.append(str(database[key][field]))

        elif command == "GET":
            if key in database and field in database[key]:
                results.append(str(database[key][field]))
            else:
                if key not in database:
                    if field is not None:
                        results.append(f"key: {key} and field: {field} do not exist")
                    else:
                        results.append(f"key: {key} does not exist")

                elif field not in database[key]:
                    results.append(f"key: {key} exists and field: {field} does not exist")

        elif command == "DELETE":
            if key in database and field in database[key]:
                del database[key][field]
                if not database[key]:
                    del database[key]
                results.append("true")
            else:
                results.append("false")

    return results

# Example usage:
queries = [
    ["SET_OR_INCREMENT", "employee1", "age", "20"],
    ["SET_OR_INCREMENT", "employee2", "age", "21"],
    ["GET", "employee1", "age"],
    ["GET", "employee3", "age"],
    ["SET_OR_INCREMENT", "employee3", "age", "32"],
    ["GET", "employee3", "age"],
]

print(inMemory_database_query(queries))  # Expected output: ["20", "21", "20"]

queries = [
    ["GET", "dept4", "floors"],
    ["GET", "dept4", "floors"],
    ["SET_OR_INCREMENT", "dept4", "floors", "22"],
    ["GET", "dept4", "stage"],
    ["GET", "dept4", "floors"],
    ["GET", "dept4", "floors"],
    ["SET_OR_INCREMENT", "dept4", "floors", "4"],
    ["GET", "dept4", "floors"],
    ["SET_OR_INCREMENT", "dept4", "floors", "10"],
    ["GET", "dept4", "floors"]
]

print(inMemory_database_query(queries))  # Expected return value: ["", "", "22", "22", "22", "26", "26", "36", "36"]

['20', '21', '20', 'key: employee3 and field: age do not exist', '32', '32']
['key: dept4 and field: floors do not exist', 'key: dept4 and field: floors do not exist', '22', 'key: dept4 exists and field: stage does not exist', '22', '22', '26', '26', '36', '36']


In [2]:
class InMemoryDatabase:
    def __init__(self):
        self.data = {}
        self.mod_count = {}

    def set_or_inc(self, key, field, value):
        if key not in self.data:
            self.data[key] = {}
            self.mod_count[key] = 0

        if field in self.data[key]:
            self.data[key][field] += int(value)
        else:
            self.data[key][field] = int(value)

        self.mod_count[key] += 1
        return str(self.data[key][field])

    def get(self, key, field):
        if key in self.data and field in self.data[key]:
            return str(self.data[key][field])
        else:
            return 'NULL'

    def delete(self, key, field):
        if key in self.data and field in self.data[key]:
            del self.data[key][field]
            self.mod_count[key] += 1
            if not self.data[key]:
                del self.data[key]
                del self.mod_count[key]
            return 'true'
        else:
            return 'false'

    def num_equal_to(self, value):
        count = 0
        for key in self.data:
            count += list(self.data[key].values()).count(int(value))
        return str(count)

    def sort_key(self, item):
        key, count = item
        return (-count, key)

    def top_n_keys(self, n):
        sorted_keys = sorted(self.mod_count.items(), key=self.sort_key)
        result = [f"{key}({count})" for key, count in sorted_keys[:n]]
        if len(result) == 1:
            result = result[0]
        return result

    #def top_n_keys(self, n):
    #    sorted_keys = sorted(self.mod_count.items(), key=lambda x: (-x[1], x[0]))
    #    result = [f"{key}({count})" for key, count in sorted_keys[:n]]
    #    if len(result) == 1:
    #        result = result[0]
    #    return result

    def execute_query(self, query):
        operation = query[0]
        if operation == "SET_OR_INCREMENT":
            return self.set_or_inc(query[1], query[2], query[3])
        elif operation == "GET":
            return self.get(query[1], query[2])
        elif operation == "DELETE":
            return self.delete(query[1], query[2])
        elif operation == "NUM_EQUAL_TO":
            return self.num_equal_to(query[1])
        elif operation == "TOP_N_KEYS":
            return self.top_n_keys(int(query[1]))
        else:
            return 'INVALID OPERATION'

    def process_queries(self, queries):
        result = []
        for query in queries:
            result.append(self.execute_query(query))
        return result

# Example usage and test case
queries = [
    ["SET_OR_INCREMENT", "employee", "age", "20"],
    ["SET_OR_INCREMENT", "employee", "id", "234"],
    ["TOP_N_KEYS", "2"]
]

db = InMemoryDatabase()
output = db.process_queries(queries)
print(output)

# Example usage and test case
queries = [
    ["SET_OR_INCREMENT", "C", "field1", "10"],
    ["TOP_N_KEYS", "5"],
    ["SET_OR_INCREMENT", "A", "field1", "5"],
    ["DELETE", "A", "field1"],
    ["SET_OR_INCREMENT", "B", "field1", "8"],
    ["SET_OR_INCREMENT", "B", "field1", "0"],
    ["TOP_N_KEYS", "3"],
    ["DELETE", "B", "field2"],
    ["SET_OR_INCREMENT", "B", "field2", "6"],
    ["SET_OR_INCREMENT", "A", "field1", "4"],
    ["SET_OR_INCREMENT", "A", "field1", "2"],
    ["TOP_N_KEYS", "2"],
    ["TOP_N_KEYS", "5"]
]

db = InMemoryDatabase()
output = db.process_queries(queries)
print(output)

['20', '234', 'employee(2)']
['10', 'C(1)', '5', 'true', '8', '8', ['B(2)', 'C(1)'], 'false', '6', '4', '6', ['B(3)', 'A(2)'], ['B(3)', 'A(2)', 'C(1)']]


In [3]:
# Record and hold transactions (deposits, withdrawals and transfers).

from datetime import datetime

class Transaction:
    def __init__(self, transaction_id, amount, timestamp, transaction_type, account_id):
        self.transaction_id = transaction_id
        self.amount = amount
        self.timestamp = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
        self.transaction_type = transaction_type
        self.account_id = account_id

    def __str__(self):
        return (f"Transaction(id={self.transaction_id}, amount={self.amount}, "
                f"timestamp={self.timestamp}, type={self.transaction_type}, "
                f"account_id={self.account_id})")

    def __repr__(self):
        return (f"Transaction(transaction_id={self.transaction_id}, amount={self.amount}, "
                f"timestamp={self.timestamp}, type={self.transaction_type}, account_id={self.account_id})")


class Account:
    def __init__(self, account_id):
        self.account_id = account_id
        self.balance = 0.0
        self.transactions = []

    def deposit(self, transaction):
        if transaction.transaction_type == 'deposit' and transaction.account_id == self.account_id:
            self.balance += transaction.amount
            self.transactions.append(transaction)

    def withdraw(self, transaction):
        if transaction.transaction_type == 'withdraw' or \
            transaction.transaction_type == 'transfer'and \
            transaction.account_id == self.account_id:
            self.balance -= transaction.amount
            self.transactions.append(transaction)

    def get_transaction_history(self):
        return self.transactions

    def get_balance(self):
        return self.balance

class Bank:
    def __init__(self):
        self.accounts = {}

    def create_account(self, account_id):
        if account_id not in self.accounts:
            self.accounts[account_id] = Account(account_id)
        else:
            print(f"create_account failed: Account {account_id} exists.")

        return self.accounts[account_id]

    def get_account(self, account_id):
        return self.accounts.get(account_id)

    def execute_transaction(self, transaction):        #
        account = self.get_account(transaction.account_id)
        if transaction.transaction_type == 'deposit':
            account.deposit(transaction)
        # elif transaction.transaction_type == 'transfer':
        elif transaction.transaction_type == 'withdraw' or \
            transaction.transaction_type == 'transfer':
            account.withdraw(transaction)

    def check_valid_account(self, account, account_id):
        if account is None:
            print(f"Transfer failed: Account {account_id} does not exist.")
            return False
        else:
            return True

    # this is a special execution that the bank does the atomic action of both deposit and withdraw between 2 or more accounts
    def transfer(self, from_account_id, to_account_id, amount, transaction_id, timestamp):
        from_account = self.get_account(from_account_id)
        to_account = self.get_account(to_account_id)
        """
        # Pre-condition: Validity check
        if (self.check_valid_account(from_account, from_account_id)) == False:
            return None

        if (self.check_valid_account(to_account, to_account_id)) == False:
            return None

        if from_account.balance < amount:
            print(f"Transfer failed: Insufficient funds in account {from_account_id}.")
            return
        """
        if from_account and to_account and from_account.balance >= amount:
            withdraw_transaction = Transaction(transaction_id=transaction_id, amount=amount,
                                               timestamp=timestamp, transaction_type='transfer',
                                               account_id=from_account_id)
            deposit_transaction = Transaction(transaction_id=transaction_id, amount=amount,
                                              timestamp=timestamp, transaction_type='deposit',
                                              account_id=to_account_id)
            from_account.withdraw(withdraw_transaction)
            to_account.deposit(deposit_transaction)

# Example Usage for Bank Class
bank = Bank()
account123 = bank.create_account(123)
account456 = bank.create_account(456)
account456_Sybil = bank.create_account(456)

transaction1 = Transaction(transaction_id=1, amount=200.0, timestamp='2024-06-25 10:00:00', transaction_type='deposit', account_id=123)
bank.execute_transaction(transaction1)
bank.transfer(from_account_id=123, to_account_id=456, amount=50.0, transaction_id=2, timestamp='2024-06-25 11:00:00')
transaction2 = Transaction(transaction_id=2, amount=50.0, timestamp='2024-06-25 12:00:00', transaction_type='withdraw', account_id=123)
bank.execute_transaction(transaction2)

bank.transfer(from_account_id=456, to_account_id=123, amount=10.0, transaction_id=3, timestamp='2024-06-25 13:00:00')
bank.transfer(from_account_id=456, to_account_id=123, amount=10.0, transaction_id=4, timestamp='2024-06-25 14:00:00')

transaction_history123 = account123.get_transaction_history()
for transaction in transaction_history123:
    print(f'transaction_history for account 123: {transaction}')

transaction_history456 = account456.get_transaction_history()
for transaction in transaction_history456:
    print(f'transaction_history for account 456: {transaction}')

# Print balance and transaction history
balance123 = account123.get_balance()
print(f'Balance for account 123: {balance123}')

balance456 = account456.get_balance()
print(f'Balance for account 456: {balance456}')


create_account failed: Account 456 exists.
transaction_history for account 123: Transaction(id=1, amount=200.0, timestamp=2024-06-25 10:00:00, type=deposit, account_id=123)
transaction_history for account 123: Transaction(id=2, amount=50.0, timestamp=2024-06-25 11:00:00, type=transfer, account_id=123)
transaction_history for account 123: Transaction(id=2, amount=50.0, timestamp=2024-06-25 12:00:00, type=withdraw, account_id=123)
transaction_history for account 123: Transaction(id=3, amount=10.0, timestamp=2024-06-25 13:00:00, type=deposit, account_id=123)
transaction_history for account 123: Transaction(id=4, amount=10.0, timestamp=2024-06-25 14:00:00, type=deposit, account_id=123)
transaction_history for account 456: Transaction(id=2, amount=50.0, timestamp=2024-06-25 11:00:00, type=deposit, account_id=456)
transaction_history for account 456: Transaction(id=3, amount=10.0, timestamp=2024-06-25 13:00:00, type=transfer, account_id=456)
transaction_history for account 456: Transaction(i

In [5]:
# Data metrics, returning the top k accounts with outgoing money

class BankAnalytics:
    def __init__(self, accounts):
        self.accounts = accounts

    def account_with_outgoing_money(self, k):
        account_outgoing = {}
        for account in self.accounts.values():
            outgoing_sum = sum(t.amount for t in account.get_transaction_history() if t.transaction_type == 'transfer')
            account_outgoing[account.account_id] = outgoing_sum
        sorted_accounts = sorted(account_outgoing.items(), key=lambda item: item[1], reverse=True)
        print(f'sorted_accounts: {sorted_accounts}')
        return sorted_accounts[:k]

    def get_account_with_highest_amount_of_outgoing_money(self, k):
        accounts = self.account_with_outgoing_money(k)
        amount_highest = 0
        id_with_amount_highest = None

        print("Top accounts with outgoing money>:")
        for account_id, amount in accounts:
            if amount >= amount_highest:
                amount_highest = amount
                id_with_amount_highest = account_id

        print(f"Account ID: {id_with_amount_highest}, has the highest Outgoing Amount: {amount_highest}")
        return (id_with_amount_highest, amount_highest)

# Example usage
accounts = {123: account123}  # Dictionary of Account objects
analytics = BankAnalytics(accounts)
top_accounts = analytics.account_with_outgoing_money(5)
print(top_accounts)

accounts = {456: account456}  # Dictionary of Account objects
analytics = BankAnalytics(accounts)
top_accounts = analytics.account_with_outgoing_money(5)
print(top_accounts)

# Analytics for accounts
accounts = {123: account123, 456: account456}  # Dictionary of Account objects
analytics = BankAnalytics(accounts)

# Query for top accounts with outgoing money
top_account = analytics.get_account_with_highest_amount_of_outgoing_money(5)
print(f"top_account: {top_account}")

# Example with separate queries (stacked)
accounts1 = {123: account123}
analytics1 = BankAnalytics(accounts1)
top_accounts1 = analytics1.account_with_outgoing_money(5)
print("Top accounts with outgoing money (query 1):")
for account_id, amount in top_accounts1:
    print(f"Account ID: {account_id}, Outgoing Amount: {amount}")

accounts2 = {456: account456}
analytics2 = BankAnalytics(accounts2)
top_accounts2 = analytics2.account_with_outgoing_money(5)
print("Top accounts with outgoing money (query 2):")
for account_id, amount in top_accounts2:
    print(f"Account ID: {account_id}, Outgoing Amount: {amount}")

# Compare the results
print("\nComparing top accounts from separate queries:")
for (acc_id1, amount1), (acc_id2, amount2) in zip(top_accounts1, top_accounts2):
    if amount1 > amount2:
        print(f"Account ID: {acc_id1} has more outgoing money: {amount1}")
    else:
        print(f"Account ID: {acc_id2} has more outgoing money: {amount2}")

sorted_accounts: [(123, 0)]
[(123, 0)]
sorted_accounts: [(456, 20.0)]
[(456, 20.0)]
sorted_accounts: [(456, 20.0), (123, 0)]
Top accounts with outgoing money>:
Account ID: 456, has the highest Outgoing Amount: 20.0
top_account: (456, 20.0)
sorted_accounts: [(123, 0)]
Top accounts with outgoing money (query 1):
Account ID: 123, Outgoing Amount: 0
sorted_accounts: [(456, 20.0)]
Top accounts with outgoing money (query 2):
Account ID: 456, Outgoing Amount: 20.0

Comparing top accounts from separate queries:
Account ID: 456 has more outgoing money: 20.0


In [6]:
# Scheduled transactions and cancellation mechanism

from datetime import datetime, timedelta
import time

class ScheduledTransaction(Transaction):
    """
    def __init__(self, transaction_id, amount, timestamp, transaction_type, account_id, scheduled_time):
        super().__init__(transaction_id, amount, timestamp, transaction_type, account_id)
        self.scheduled_time = datetime.strptime(scheduled_time, '%Y-%m-%d %H:%M:%S')
        self.is_scheduled = False
    """

    def __init__(self, transaction_id, amount, timestamp, transaction_type, account_id, scheduled_time, target_account_id=None):
        super().__init__(transaction_id, amount, timestamp, transaction_type, account_id)
        self.scheduled_time = datetime.strptime(scheduled_time, '%Y-%m-%d %H:%M:%S')
        self.is_scheduled = False
        self.target_account_id = target_account_id  # Target account for transfers (i.e. to_account_id)

    def schedule_transaction(self):
        self.is_scheduled = True
        print(f"Transaction {self.transaction_id} scheduled for {self.scheduled_time}")

    def cancel_transaction(self):
        self.is_scheduled = False
        print(f"Transaction {self.transaction_id} canceled")

    def time_until_scheduled(self):
        now = datetime.now()
        time_diff = (self.scheduled_time - now).total_seconds()

        # check time diff. Caveat: check if the schedule time is before `now` (diff will be -ve)
        assert time_diff >= 0, f'{self.scheduled_time} is before {now} by {abs(time_diff)} secs'

        return max(time_diff, 0)

    def trigger_action(self, bank):
        if self.is_scheduled and self.time_until_scheduled() == 0:
            if self.transaction_type == 'transfer' and self.target_account_id:
                bank.transfer(from_account_id=self.account_id, to_account_id=self.target_account_id,
                              amount=self.amount, transaction_id=self.transaction_id,
                              timestamp=self.scheduled_time.strftime('%Y-%m-%d %H:%M:%S'))
            else:   # atomic actions: deposit or withdraw
                bank.record_transaction(self)
            print(f"Transaction {self.transaction_id} executed at {self.scheduled_time}")

# Example usage
#scheduled_transaction = ScheduledTransaction(transaction_id=3, amount=200.0, timestamp='2024-06-26 09:00:00', transaction_type='deposit', account_id=456, scheduled_time='2024-06-20 09:00:00')
scheduled_transaction = ScheduledTransaction(transaction_id=3, amount=200.0, timestamp='2024-06-26 09:00:00', transaction_type='deposit', account_id=456, scheduled_time='2024-06-26 09:00:00')
scheduled_transaction.schedule_transaction()

#scheduled_transfer = ScheduledTransaction(transaction_id=2, amount=50.0, timestamp='2024-06-25 11:00:00',
#                                          transaction_type='transfer', account_id=123,
#                                          scheduled_time='2024-06-25 11:00:00', target_account_id=456)
#scheduled_transfer.schedule_transaction()
"""
scheduled_transaction.schedule_transaction()
# scheduled_transaction.cancel_transaction()

time_diff = scheduled_transaction.time_until_scheduled()
print(f"Time until scheduled transaction: {time_diff} seconds")

# Assuming we have the Bank class from the previous examples
#bank = Bank()
#account456 = bank.create_account(456)

# Simulating time passage for the scheduled transaction to be triggered
#time.sleep(time_diff)  # Wait until the scheduled time
#scheduled_transaction.trigger_action(bank)
"""
transaction1 = Transaction(transaction_id=1, amount=200.0, timestamp='2024-06-25 10:00:00', transaction_type='deposit', account_id=123)
bank.execute_transaction(transaction1)
bank.transfer(from_account_id=123, to_account_id=456, amount=50.0, transaction_id=2, timestamp='2024-06-25 11:00:00')
transaction2 = Transaction(transaction_id=2, amount=50.0, timestamp='2024-06-25 12:00:00', transaction_type='withdraw', account_id=123)
bank.execute_transaction(transaction2)

# Check if the transaction was executed
balance456 = account456.get_balance()
print(f'Balance for account 456 after scheduled transaction: {balance456}')
transaction_history456 = account456.get_transaction_history()
for transaction in transaction_history456:
    print(f'transaction_history for account 456: {transaction}')


Transaction 3 scheduled for 2024-06-26 09:00:00
Balance for account 456 after scheduled transaction: 80.0
transaction_history for account 456: Transaction(id=2, amount=50.0, timestamp=2024-06-25 11:00:00, type=deposit, account_id=456)
transaction_history for account 456: Transaction(id=3, amount=10.0, timestamp=2024-06-25 13:00:00, type=transfer, account_id=456)
transaction_history for account 456: Transaction(id=4, amount=10.0, timestamp=2024-06-25 14:00:00, type=transfer, account_id=456)
transaction_history for account 456: Transaction(id=2, amount=50.0, timestamp=2024-06-25 11:00:00, type=deposit, account_id=456)


In [7]:
# Merge 2 accounts while maintaining separate account histories

class AccountMerger:
    def merge_accounts(self, account1, account2):
        if account1.account_id != account2.account_id:
            merged_transactions = account1.get_transaction_history() + account2.get_transaction_history()
            merged_balance = account1.get_balance() + account2.get_balance()

            merged_account = Account(account_id=f"{account1.account_id}-{account2.account_id}")
            merged_account.transactions = merged_transactions
            merged_account.balance = merged_balance
            return merged_account

# Example usage
#account456 = Account(account_id=456)
#transaction3 = Transaction(transaction_id=3, amount=200.0, timestamp='2024-06-26 09:00:00', transaction_type='deposit', account_id=456)
#account456.deposit(transaction3)

merger = AccountMerger()
merged_account = merger.merge_accounts(account123, account456)
result = merged_account.get_transaction_history()

for i in range(0, len(result)):
    print(f'transaction_history: {result[i]}')

print(merged_account.get_balance())


transaction_history: Transaction(id=1, amount=100.0, timestamp=2024-06-25 10:00:00, type=deposit, account_id=123)
transaction_history: Transaction(id=2, amount=50.0, timestamp=2024-06-25 11:00:00, type=deposit, account_id=456)
transaction_history: Transaction(id=3, amount=10.0, timestamp=2024-06-25 13:00:00, type=transfer, account_id=456)
transaction_history: Transaction(id=4, amount=10.0, timestamp=2024-06-25 14:00:00, type=transfer, account_id=456)
transaction_history: Transaction(id=2, amount=50.0, timestamp=2024-06-25 11:00:00, type=deposit, account_id=456)
180.0
